In [1]:
import pandas as pd

In [3]:
import os

Address_df = pd.read_csv('sample_data/Address_s.csv')
ContactPoint_df = pd.read_csv('sample_data/ContactPoint_s.csv')
HealthcareOrganization_df = pd.read_csv('sample_data/HealthcareOrganization_s.csv')
HealthcarePersonnel_df = pd.read_csv('sample_data/HealthcarePersonnel_s.csv')
Person_df = pd.read_csv('sample_data/Person_s.csv')
ServiceDepartment_df = pd.read_csv('sample_data/ServiceDepartment_s.csv')

# Specify the encoding to handle the decoding error
golden_record = pd.read_csv('updated_golden_standard_duplicates.csv', encoding='latin1')

In [37]:


# contact_point = ContactPoint_df[ContactPoint_df["identifier"] == "23a7711a-8133-4876-b7eb-dcd9e87a1613"]
# language = contact_point['availableLanguage']
# language = language.str.strip("[]").str.split(',').str[0]
# print(language)

# language = language.iloc[0].strip("'")
# print(language)

# # Strip the single quotes from the language string
# language = language.strip("'")
# print(language)

In [4]:
golden_record

,original_id,duplicate_id,entity_type,variation_type,field_name,original_value,varied_value
0,0e8cff18-9e9d-4b77-ba34-b580eda0a0af,36e27309-6297-4cc9-ac4f-8c54df56c845,Address,house_number_suffix,text,Leahbaan 26,Leahbaan 26B
1,1095e497-3d79-4f0b-a6cd-3595d4f0d65d,d3d62e46-973f-43c1-b8c3-6a4839f07ec8,Address,country_expansion,country,AT,Austria
2,3899c22a-d4ed-41bd-9692-74a2acbe4ff6,bfc41ec3-5589-4c05-a0f9-a7a89e52b550,Address,house_number_suffix,text,Siim Corners 52,Siim Corners 52C
3,541d1a81-4815-442c-a2bb-62833d83ad7c,4e68583f-aa89-449e-beda-361e081a5a0f,Address,postal_format,postalCode,1625 AP,1625AP
4,603fa8f0-1549-4dee-830b-4d6d9575d095,a06c2e83-080c-4ac9-9f6e-cc909c053fc1,Address,country_expansion,country,NL,Netherlands
...,...,...,...,...,...,...,...
933,0a4d3590-f9b4-4573-878a-2aafd8dc0cb1,321237a1-dc60-4be0-a845-41188a5f28a3,ContactPoint,email_typo,email,Delgado.Public@dept.healthcare.org,Delgaado.Public@dept.healthcare.org
934,5aeb3613-9cc4-4f93-8fd9-ba8f6dc6e925,4afb4240-0c7c-4e9d-a99c-7ca625b6e719,ContactPoint,email_typo,email,Berg.Hematologic@dept.healthcare.org,Berg.Hematologc@dept.healthcare.org
935,92738995-ca89-4d11-8646-356bc28611fd,b9c9c809-40a2-4ca3-94f3-d4eb6f5be493,ContactPoint,email_typo,email,Jordan.Gastroenterologic@dept.healthcare.org,Jordan.Gstroenterologic@dept.healthcare.org
936,51b0a3c5-c9e7-4aba-b5da-3e50822bbc3d,718eff2d-4298-4c03-bec2-df5c45514bb5,ContactPoint,email_domain_change,email,Delgado.Dentistry@dept.healthcare.org,Delgado.Dentistry@dept.healthcare.nl


In [5]:
# Dictionary to store all dataframes by entity type
entity_dataframes = {}

# Get a static list of variable names first to avoid the iteration error
variable_names = list(locals().keys())

# Populate the dictionary with your existing dataframes
for var_name in variable_names:
    if var_name.endswith('_df'):
        entity_type = var_name.replace('_df', '')
        entity_dataframes[entity_type] = locals()[var_name]

# Iterate through each row in the golden record
for _, row in golden_record.iterrows():
    entity_type = row['entity_type']
    
    # Check if we have a dataframe for this entity type
    if entity_type in entity_dataframes:
        df = entity_dataframes[entity_type]
        
        # Find the matching row using original_id
        mask = df['identifier'] == row['original_id']
        
        # If a match is found, update the specified field with the varied value
        if mask.any():
            field_name = row['field_name']
            if field_name in df.columns:
                df.loc[mask, field_name] = row['varied_value']
                print(f"Updated {entity_type} - {row['original_id']} - {field_name}: {row['varied_value']}")
            else:
                print(f"Warning: Field {field_name} not found in {entity_type} dataframe")
        else:
            print(f"Warning: No row with identifier {row['original_id']} found in {entity_type} dataframe")
    else:
        print(f"Warning: No dataframe found for entity type {entity_type}")

Updated Address - 0e8cff18-9e9d-4b77-ba34-b580eda0a0af - text: Leahbaan 26B
Updated Address - 1095e497-3d79-4f0b-a6cd-3595d4f0d65d - country: Austria
Updated Address - 3899c22a-d4ed-41bd-9692-74a2acbe4ff6 - text: Siim Corners 52C
Updated Address - 541d1a81-4815-442c-a2bb-62833d83ad7c - postalCode: 1625AP
Updated Address - 603fa8f0-1549-4dee-830b-4d6d9575d095 - country: Netherlands
Updated Address - eedbba8b-0f83-41ee-b2ac-89bdb2998fa3 - text: Linda Expressway 107B
Updated Address - 853a9135-3ee2-451b-be88-c93319db1d7b - text: Katarina-Rohrer-Gasse 42A
Updated Address - 7c5246f1-8fe5-475b-b771-877e141e39e7 - city: Bunnschoten-Spakenburg
Updated Address - eae9c546-446c-4893-89d3-cdb9ebb9aa72 - postalCode: 6850ER
Updated Address - 100cfe80-2132-4c24-81b3-77bd59eae69a - text: Maritweg 143C
Updated Address - 335be677-df1b-425e-af66-be879a4c2227 - postalCode: 8883YA
Updated Address - 7d859725-c707-4ef9-86c3-744cc88e03b6 - text: Jordyring 13B
Updated Address - d35f581e-3648-431f-ae9e-8d038498

In [6]:
HealthcareOrganization_df

,identifier,healthcareOrganizationName,address,contactPoint
0,06d2ed7c-e6ac-4d8a-8160-ff927c7550f2,Idbarra PLC Zorg,7d859725-c707-4ef9-86c3-744cc88e03b6,ef151673-a1df-4da7-9d44-c93e7799a8e2
1,4a863655-0a9d-499c-a269-8c2d614f1ddc,"Berg, Long and Middleton Tervisekeskus",6bc9e970-c48a-4374-86dc-1579647640a1,29502326-7644-4183-9aab-d22d75ef8aab
2,037d9752-48f3-431c-8fb7-c3bb4408c04c,JJK Zorg,6756264d-c864-4f78-beaf-167952b04994,57627626-2037-4089-b11f-a10a0b2975c6
3,5d393bf9-b2da-41f6-8c16-014a2e4a6e78,PB Gesundheitszentrum,568d6f3e-f4b4-45aa-adfd-a2912fdb5c0d,fae3fcee-7636-4fd9-8376-ffa876fe579f
4,c058a332-b4cf-4fa8-ac98-f73d27fe1b4b,"Griffin, Ross and Saunders Zorg",27649a62-b02d-452c-9b05-0db28ee4fd02,74151acc-f5a3-4893-bf7a-27482cbb93c2
5,3e332f35-086c-4768-b0fe-3cfc991ab513,Nosvak PLC Tervisekeskus,99755cb9-7c76-4592-a73e-3f3b5c27761d,eec9fa67-4cc6-42fc-b006-ff85166a1a98
6,92d6ff73-5b3a-4595-8458-92c14e0e15d3,DGS Zorg,3856be31-f568-49c6-833c-14694f06ea18,b4480761-505e-4c9b-8bc8-8bcf7b1d4049
7,9db3466d-1ba6-4c61-aaf9-4919e69b7971,PMB Gesundheitszentrum,1095e497-3d79-4f0b-a6cd-3595d4f0d65d,23254d81-2bdf-4727-9501-e917496dc27b
8,30d6863f-11c2-42d8-8126-77895c6be596,Kim-Tod Gesundheitszentrum,4598244b-f0bd-4405-a50d-d400a27ee80d,4e98ce25-1733-4a7c-93e8-36639c14940d
9,71a8c9c6-0f6a-475b-b55b-2e5ca6ed0ac0,FS Zorg,8b279c29-a274-40d7-b4a5-cc36692a7bce,9177b6d4-5f2e-4167-b13e-ceb14ad12b4c


In [7]:
from faker import Faker

fake = Faker()

# Create a copy to avoid modifying original data
A_copy = Address_df.copy()
CP_copy = ContactPoint_df.copy()
P_copy = Person_df.copy()
HO_copy = HealthcareOrganization_df.copy()
SD_copy = ServiceDepartment_df.copy()
HP_copy = HealthcarePersonnel_df.copy()

# Save old values
old_valuesHO = {
    'identifier': HO_copy['identifier'].copy(),
    'address': HO_copy['address'].copy(),
    'contactPoint': HO_copy['contactPoint'].copy()
}

# Generate new UUIDs for each field
HO_copy['identifier'] = HO_copy['identifier'].apply(lambda x: fake.uuid4())
HO_copy['address'] = HO_copy['address'].apply(lambda x: fake.uuid4())
HO_copy['contactPoint'] = HO_copy['contactPoint'].apply(lambda x: fake.uuid4())

# Create copies of dataframes
old_valuesSD = {
    'identifier': SD_copy['identifier'].copy(),
    'address': SD_copy['address'].copy(),
    'contactPoint': SD_copy['contactPoint'].copy()
}

# Generate new UUIDs for each field in ServiceDepartment_df
SD_copy['identifier'] = SD_copy['identifier'].apply(lambda x: fake.uuid4())
SD_copy['address'] = SD_copy['address'].apply(lambda x: fake.uuid4())
SD_copy['contactPoint'] = SD_copy['contactPoint'].apply(lambda x: fake.uuid4())

old_valuesHP = {
    'identifier': HP_copy['identifier'].copy()
}
# Generate new UUIDs for Healthcare Personnel identifiers
HP_copy['identifier'] = HP_copy['identifier'].apply(lambda x: fake.uuid4())

In [8]:
HO_copy

,identifier,healthcareOrganizationName,address,contactPoint
0,f365566d-f533-4ae8-93be-2743d3aab509,Idbarra PLC Zorg,47d6af77-8705-4c59-ac95-a96fa9f52e8b,b15a9aaa-6a4f-4848-abce-057ed48099e3
1,6213a29d-6d9c-484d-8005-de4f81ba0102,"Berg, Long and Middleton Tervisekeskus",019329ac-a429-41fc-ae6d-e7afd8cc3a1f,aa161e70-13ae-407c-8a27-57113308cdf7
2,3d598812-bdbc-4373-aca8-5d9ab013042c,JJK Zorg,8cd26987-f9a7-4823-9c2f-e4108859dbf2,0704fc08-8911-4cb1-b449-4413ca1d3b55
3,5ff8159d-efb9-4f87-a9d8-b1d7909de685,PB Gesundheitszentrum,a2e534a2-1ae3-4981-a16d-51361091ee65,9731dba3-680a-43be-a834-54101751d33f
4,612d8f0c-7ef7-4827-955b-60e92658af19,"Griffin, Ross and Saunders Zorg",f758abe9-f534-480f-a4bc-d1dbd3a50b38,d300b45f-f44a-4ca5-8d5b-dcb936598ad0
5,e3ae9185-488e-4fcf-915f-b9fcd4c20e81,Nosvak PLC Tervisekeskus,4af53614-5cb6-4067-a92e-f8fd7a0f5d74,8da6c2a6-45ca-4b43-98cc-00abb8c0efb4
6,2a4c7e62-dba3-4a63-91bf-2cad324cc811,DGS Zorg,4da4229a-053a-41be-bb8c-cf6c7117e000,d9d8a2d9-0fca-4016-8194-b5887df15638
7,21f7f6a5-d115-49a4-975d-8d01a38a98e7,PMB Gesundheitszentrum,679dbeb2-2136-402b-811b-c2471abca0b2,f745180e-bc38-4e66-a438-fd2821c93fd0
8,4d655899-f2d2-4f8b-ac02-448ecbcb49db,Kim-Tod Gesundheitszentrum,c47b51c8-4feb-46d6-a99f-fdcbb56aaa20,bd414832-4f5c-46b2-a82f-e8847368d8e8
9,506b7960-a393-4161-8a89-7eb37ee7624d,FS Zorg,b4e4d345-209d-42fb-bf2b-3196cc360c99,8e49f0f6-78a2-4f0f-a5b7-758fdad6b5ef


In [9]:
# Create a mapping of old to new UUIDs
HO_uuid_mapping = {
    old_valuesHO['identifier'].iloc[i]: HO_copy['identifier'].iloc[i]
    for i in range(len(HO_copy))
}

# Update references in other dataframes (using the copies)
# Update institution references in Healthcare_Personnel_df copy
HP_copy['institution'] = HP_copy['institution'].map(HO_uuid_mapping).fillna(HP_copy['institution'])

# Update isPartOf references in ServiceDepartment_df copy
SD_copy['isPartOf'] = SD_copy['isPartOf'].map(HO_uuid_mapping).fillna(SD_copy['isPartOf'])

# Update contact point references
ho_contact_mapping = {
    old_valuesHO['contactPoint'].iloc[i]: HO_copy['contactPoint'].iloc[i]
    for i in range(len(HO_copy))
}

# Update identifier in ContactPoint_df copy where it matches old contact points
CP_copy['identifier'] = CP_copy['identifier'].map(ho_contact_mapping).fillna(CP_copy['identifier'])

# Update address references
ho_address_mapping = {
    old_valuesHO['address'].iloc[i]: HO_copy['address'].iloc[i]
    for i in range(len(HO_copy))
}

# Update identifier in Address_df copy where it matches old addresses
A_copy['identifier'] = A_copy['identifier'].map(ho_address_mapping).fillna(A_copy['identifier'])

# Create mapping for ServiceDepartment
sd_uuid_mapping = {
    old_valuesSD['identifier'].iloc[i]: SD_copy['identifier'].iloc[i]
    for i in range(len(SD_copy))
}

# Create address mapping for ServiceDepartment
sd_address_mapping = {
    old_valuesSD['address'].iloc[i]: SD_copy['address'].iloc[i]
    for i in range(len(SD_copy))
}

# Create contact point mapping for ServiceDepartment
sd_contact_mapping = {
    old_valuesSD['contactPoint'].iloc[i]: SD_copy['contactPoint'].iloc[i]
    for i in range(len(SD_copy))
}

# Update references in Address_df copy for ServiceDepartment
A_copy['identifier'] = A_copy['identifier'].map(sd_address_mapping).fillna(A_copy['identifier'])

# Update references in ContactPoint_df copy for ServiceDepartment
CP_copy['identifier'] = CP_copy['identifier'].map(sd_contact_mapping).fillna(CP_copy['identifier'])

# Create mapping for Personnel
hp_uuid_mapping = {
    old_valuesHP['identifier'].iloc[i]: HP_copy['identifier'].iloc[i]
    for i in range(len(HP_copy))
}

# Update references in Person_df copy
P_copy['identifier'] = P_copy['identifier'].map(hp_uuid_mapping).fillna(P_copy['identifier'])

In [10]:
old_valuesHO

{'identifier': 0     06d2ed7c-e6ac-4d8a-8160-ff927c7550f2
 1     4a863655-0a9d-499c-a269-8c2d614f1ddc
 2     037d9752-48f3-431c-8fb7-c3bb4408c04c
 3     5d393bf9-b2da-41f6-8c16-014a2e4a6e78
 4     c058a332-b4cf-4fa8-ac98-f73d27fe1b4b
 5     3e332f35-086c-4768-b0fe-3cfc991ab513
 6     92d6ff73-5b3a-4595-8458-92c14e0e15d3
 7     9db3466d-1ba6-4c61-aaf9-4919e69b7971
 8     30d6863f-11c2-42d8-8126-77895c6be596
 9     71a8c9c6-0f6a-475b-b55b-2e5ca6ed0ac0
 10    52ebdac5-a145-4899-a1f8-c1569e0df45b
 11    8b5885ca-0bb2-43f0-bd30-291a55fea08e
 12    c7405255-25cf-41c4-9c89-929692b035e0
 13    27d99a23-e4f7-425e-afe6-790abc18a40b
 14    a59cec98-126c-4c8f-b888-447911ebcd49
 Name: identifier, dtype: object,
 'address': 0     7d859725-c707-4ef9-86c3-744cc88e03b6
 1     6bc9e970-c48a-4374-86dc-1579647640a1
 2     6756264d-c864-4f78-beaf-167952b04994
 3     568d6f3e-f4b4-45aa-adfd-a2912fdb5c0d
 4     27649a62-b02d-452c-9b05-0db28ee4fd02
 5     99755cb9-7c76-4592-a73e-3f3b5c27761d
 6     3856be31-f

In [11]:
import os

# Create the new directory if it doesn't exist
if not os.path.exists('testdata_dup'):
    os.makedirs('testdata_dup')

# Save each dataframe to CSV in the new directory
A_copy.to_csv('sample_dup/address_d.csv', index=False)
CP_copy.to_csv('sample_dup/ContactPoint_d.csv', index=False)
HO_copy.to_csv('sample_dup/HealthcareOrganization_d.csv', index=False)
HP_copy.to_csv('sample_dup/HealthcarePersonnel_d.csv', index=False)
P_copy.to_csv('sample_dup/Person_d.csv', index=False)
SD_copy.to_csv('sample_dup/ServiceDepartment_d.csv', index=False)

In [12]:
import pandas as pd
import os

# Define paths
original_data_path = 'sample_data'
duplicate_data_path = 'sample_dup'
golden_standard_path = 'updated_golden_standard_duplicates.csv'
output_path = 'updated_golden_standard_duplicates2.csv'

# Read golden standard file
golden_df = pd.read_csv(golden_standard_path, encoding='latin1')

# Dictionary to store matching field values between original and duplicate datasets
field_value_to_id_mapping = {}

# Process each entity type
for _, row in golden_df.iterrows():
    entity_type = row['entity_type']
    field_name = row['field_name']
    varied_value = row['varied_value']
    
    # Define file paths based on entity type
    if entity_type == 'Address':
        original_file = 'address.csv'
        duplicate_file = 'address_d.csv'
    elif entity_type == 'ContactPoint':
        original_file = 'ContactPoint.csv'
        duplicate_file = 'ContactPoint_d.csv'
    elif entity_type == 'HealthcareOrganization':
        original_file = 'HealthcareOrganization.csv'
        duplicate_file = 'HealthcareOrganization_d.csv'
    elif entity_type == 'HealthcarePersonnel':
        original_file = 'HealthcarePersonnel.csv'
        duplicate_file = 'HealthcarePersonnel_d.csv'
    elif entity_type == 'Person':
        original_file = 'Person.csv'
        duplicate_file = 'Person_d.csv'
    elif entity_type == 'ServiceDepartment':
        original_file = 'ServiceDepartment.csv'
        duplicate_file = 'ServiceDepartment_d.csv'
    else:
        continue
    
    # Read duplicate dataset
    duplicate_df = pd.read_csv(os.path.join(duplicate_data_path, duplicate_file))
    
    # Look for records in the duplicate dataset that have the varied value
    # in the specified field
    if field_name in duplicate_df.columns:
        matching_rows = duplicate_df[duplicate_df[field_name] == varied_value]
        
        if not matching_rows.empty:
            # Get the identifier from the first matching row
            new_duplicate_id = matching_rows.iloc[0]['identifier']
            
            # Store the mapping for this specific row
            key = (entity_type, field_name, varied_value)
            field_value_to_id_mapping[key] = new_duplicate_id

# Create a new dataframe for the updated golden standard
updated_golden_df = golden_df.copy()

# Update the duplicate_id column based on our mappings
for i, row in updated_golden_df.iterrows():
    key = (row['entity_type'], row['field_name'], row['varied_value'])
    if key in field_value_to_id_mapping:
        updated_golden_df.at[i, 'duplicate_id'] = field_value_to_id_mapping[key]

# Save the updated golden standard file
updated_golden_df.to_csv(output_path, index=False, encoding='utf-8')

print(f"Updated golden standard file saved to {output_path}")
print(f"Total field-value mappings created: {len(field_value_to_id_mapping)}")
print(f"Duplicate IDs updated: {sum(updated_golden_df['duplicate_id'] != golden_df['duplicate_id'])}")

Updated golden standard file saved to updated_golden_standard_duplicates2.csv
Total field-value mappings created: 840
Duplicate IDs updated: 938
